# The SLAM algorithm

## Introduction

This tuto is intended to help you organize all the pieces of the SLAM system into a functioning algorithm.

The idea of the algorithm is the following:

- Images will arrive
- For each image, you will define a keyframe, storing position and orientation of the camera
- For each keyframe, you need to link it to the previous one via a motion factor
- You have to analyse the images and detect AprilTags on them
- For each detected Tag, you will define a landmark
    - Sometimes, the tag is already known: you need to identify it and create a factor
    - Sometimes, the detected tag is new: you need to create a landmark and a factor

The following is one of the many ways you can organize your code to achieve this.

We advance n iterations:

- First iteration:  basic algorithm
- Secon iteration:  bootstrap the 1st image
- Third iteration:  set landmark warm-start values
- Fourth iteration: add a keyframe prior
- Fifth iteration:  add a landmark prior

## First iteration: basic algorithm

The basic algorithm can be put in pseudo-code as follows

In [ ]:
# AprilTag detector
detector = apriltag.Detector()

# Dictionaries for SLAM objects
keyframes = dict()
landmarks = dict()
factors   = dict()

# Time and IDs
t       = 0
kf_id   = 0
fac_id  = 0



while(True):

    # Read image
    image = cv2.imread('imagefile(t)', cv2.IMREAD_GRAYSCALE)
    if image is None: 
        break

    # make a KF for this image
    kf_id += 1
    keyframe = makeKeyFrame(...)  # see tuto_gslam for info!
    keyframes[kf_id] = keyframe

    # make a motion factor from last KF
    motion_measurement = ...
    motion_sqrt_info = np.eye(6) / 1e3
    factor = makeFactor('motion', fac_id, kf_last_id, kf_id, motion_measurement, motion_sqrt_info)  # see tuto_gslam for info!
    factors[fac_id] = factor 

    # analyze image
    detections = detector.detect(image)

    for detection in detections:

        # obtain 3d pose of tag wrt. camera
        measurement = computePose(detection.corners)  # see tuto_apriltag for info!
        sqrt_info   = np.eye(6) / 1e-2

        # see if lmk is known or new
        lmk_id = detection.tag_id
        if lmk_id in landmarks: # lmk known: we only need to add a factor

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)  # see tuto_gslam for info!
            factors[fac_id] = factor

        else: # lmk new: we need to add the new landmark, and a factor

            landmark = makeLandmark(lmk_id, ...)  # see tuto_gslam for info!
            landmarks[lmk_id] = landmark

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)  # see tuto_gslam for info!
            factors[fac_id] = factor

    # solve optimization problem!
    solveProblem(...)

    # draw all objects in 3d!
    drawAllGraphics(keyframes, landmarks, factors)

    kf_last_id = kf_id
    t += 1

# print final results!
printResults(...)


## Improvements

The algorithm above shows two important limitations:

- The first time, there is no `last keyframe` to add a motion factor
- keyframes and landmarks do not have warm-start values: the solver will take long to converge, and may diverge!
- There is no prior or absolute information to anchor the produced map to any particular position / orientation

These concerns are treated in the following sections.


### Second iteration: bootstrapping

We need to tackle the slightly different situation of the first keyframe, since there is no last keyframe to refer any motion to.

We need a marker to indicate that we just entered the problem. We use a boolean `first_time` for this.

In [ ]:
# AprilTag detector
detector = apriltag.Detector()

# Dictionaries for SLAM objects
keyframes = dict()
landmarks = dict()
factors   = dict()

# Time and IDs
t       = 0
kf_id   = 0
fac_id  = 0

# Mark first time execution
first_time = True


while(True):

    # Read image
    image = cv2.imread('imagefile(t)', cv2.IMREAD_GRAYSCALE)
    if image is None: 
        break

    # make a KF for this image
    kf_id += 1
    keyframe = makeKeyFrame(...)
    keyframes[kf_id] = keyframe

    # make a motion factor from last KF
    if not first_time:
        motion_measurement = ...
        motion_sqrt_info = np.eye(6) / 1e3
        factor = makeFactor('motion', fac_id, kf_last_id, kf_id, motion_measurement, motion_sqrt_info)
        factors[fac_id] = factor 

    # analyze image
    detections = detector.detect(image)

    for detection in detections:

        # obtain 3d pose of tag wrt. camera
        measurement = computePose(detection.corners)
        sqrt_info = np.eye(6) / 1e-2

        # see if lmk is known or new
        lmk_id = detection.tag_id
        if lmk_id in landmarks: # lmk known: we only need to add a factor

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)
            factors[fac_id] = factor

        else: # lmk new: we need to add the new landmark, and a factor

            landmark = makeLandmark(lmk_id, ...)
            landmarks[lmk_id] = landmark

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)
            factors[fac_id] = factor

    # solve optimization problem!
    solveProblem(...)

    # draw all objects in 3d!
    drawAllGraphics(keyframes, landmarks, factors)

    kf_last_id = kf_id
    t += 1

# print final results!
printResults(...)


### Third iteration: set keyframe and landmark warm-start values

In order for the solver to converge quickly, it is important in SLAM to use the semsor measurements in our favor.

In particular, we want to compute warm-start values for each new state that we add to the system.

- For keyframes, we do so by copying the values of the old keyframe into the new keyframe

- For landmarks, we use the Pose3d information in the measurement to compute the position of the new landmark

In [ ]:
# AprilTag detector
detector = apriltag.Detector()

# Dictionaries for SLAM objects
keyframes = dict()
landmarks = dict()
factors   = dict()

# Time and IDs
t       = 0
kf_id   = 0
fac_id  = 0

# Mark first time execution
first_time = True


while(True):

    # Read image
    image = cv2.imread('imagefile(t)', cv2.IMREAD_GRAYSCALE)
    if image is None: 
        break

    if first_time:
        # make a KF for this image
        kf_pos = np.array([0,0,0])
        kf_ori = np.array([0,0,0])
        keyframe = makeKeyFrame(kf_id, kf_pos, kf_ori)  # specify warm-start values!!
        keyframes[kf_id] = keyframe

    else:
        # make a KF for this image
        kf_id += 1
        kf_pos = keyframes[kf_last_id].position
        kf_ori = keyframes[kf_last_id].anglevector
        keyframe = makeKeyFrame(kf_id, kf_pos, kf_ori)  # specify warm-start values!!
        keyframes[kf_id] = keyframe

        # make a motion factor from last KF
        motion_measurement = np.array([0,0,0,  0,0,0]) # we use a constant-position motion model
        motion_sqrt_info = np.eye(6) / 1e3  # very unprecise!! this allows the solver to move this KF away from the last one
        factor = makeFactor('motion', fac_id, kf_last_id, kf_id, motion_measurement, motion_sqrt_info)
        factors[fac_id] = factor 


    # analyze image
    detections = detector.detect(image)

    for detection in detections:

        # obtain 3d pose of tag wrt. camera
        measurement = computePose(detection.corners)
        sqrt_info = np.eye(6) / 1e-2

        # see if lmk is known or new
        lmk_id = detection.tag_id
        if lmk_id in landmarks: # lmk known: we only need to add a factor

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)
            factors[fac_id] = factor

        else: # lmk new: we need to add the new landmark, and a factor

            # landmark warm-start!!
            # compose kf pose with measurement of tag pose, to obtain tag pose in global frame
            lmk_pos, lmk_ori = composePoses(lmk_pos, lmk_ori, measurement)

            landmark = makeLandmark(lmk_id, lmk_pos, lmk_ori)
            landmarks[lmk_id] = landmark

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, measurement, sqrt_info)
            factors[fac_id] = factor

    # solve optimization problem!
    solveProblem(...)

    # draw all objects in 3d!
    drawAllGraphics(keyframes, landmarks, factors)

    kf_last_id = kf_id
    t += 1

# print final results!
printResults(...)


### Fourth iteration: adding a keyframe prior

The map produced by the algorithm above only contains relative information:
- between consecutive keyframes
- from keyframes to kandmarks

It is pertinent to anchor the problem to some absolute value. For this, we can create a factor that will atract the first keyframe towards a user-defined value.

This factor will be added to the first keyframe, using the `first_time` marker we defined above:

In [ ]:
# AprilTag detector
detector = apriltag.Detector()

# Dictionaries for SLAM objects
keyframes = dict()
landmarks = dict()
factors   = dict()

# Time and IDs
t       = 0
kf_id   = 0
fac_id  = 0

# Mark first time execution
first_time = True


while(True):

    # Read image
    image = cv2.imread('imagefile(t)', cv2.IMREAD_GRAYSCALE)
    if image is None: 
        break

    # make a KF for this image
    kf_id += 1
    keyframe = makeKeyFrame(...)
    keyframes[kf_id] = keyframe

    if first_time:
        # make a KF for this image
        kf_pos = np.array([0,0,0])
        kf_ori = np.array([0,0,0])
        keyframe = makeKeyFrame(kf_id, kf_pos, kf_ori)  # specify warm-start values!!
        keyframes[kf_id] = keyframe

        # add a prior keyframe factor
        kf_measurement = np.array([0,0,0,  0,0,0])   # origin of coordinates -- same as warm-start values!!
        kf_sqrt_info = np.eye(6) / 1e6               # very precise!! 
        factor = makeFactor('prior_keyframe', fac_id, kf_id, prior_measurement, prior_sqrt_info)
        factors[fac_id] = factor

    else:
        # make a KF for this image
        kf_id += 1
        kf_pos = keyframes[kf_last_id].position
        kf_ori = keyframes[kf_last_id].anglevector
        keyframe = makeKeyFrame(kf_id, kf_pos, kf_ori)  # specify warm-start values!!
        keyframes[kf_id] = keyframe

        # make a motion factor from last KF
        motion_measurement = np.array([0,0,0,  0,0,0]) # we use a constant-position motion model
        motion_sqrt_info = np.eye(6) / 1e3  # very unprecise!! this allows the solver to move this KF away from the last one
        factor = makeFactor('motion', fac_id, kf_last_id, kf_id, motion_measurement, motion_sqrt_info)
        factors[fac_id] = factor 

    # analyze image
    detections = detector.detect(image)

    for detection in detections:

        # obtain 3d pose of tag wrt. camera
        lmk_measurement = computePose(detection.corners)
        lmk_sqrt_info = np.eye(6) / 1e-2

        # see if lmk is known or new
        lmk_id = detection.tag_id
        if lmk_id in landmarks: # lmk known: we only need to add a factor

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, lmk_measurement, lmk_sqrt_info)
            factors[fac_id] = factor

        else: # lmk new: we need to add the new landmark, and a factor

            # compose kf pose with measurement of tag pose, to obtain tag pose in global frame
            lmk_pos, lmk_ori = composePoses(lmk_pos, lmk_ori, measurement)

            landmark = makeLandmark(lmk_id, ...)
            landmarks[lmk_id] = landmark

            fac_id += 1
            factor = makeFactor('landmark', fac_id, kf_id, lmk_id, lmk_measurement, lmk_sqrt_info)
            factors[fac_id] = factor

    # solve optimization problem!
    solveProblem(...)

    # draw all objects in 3d!
    drawAllGraphics(keyframes, landmarks, factors)

    kf_last_id = kf_id
    t += 1

# print final results!
printResults(...)


### Fifth iteration: adding a landmark prior

Adding a landmark prior is more complicated since doing so impacts a whole number of things, in particular the way we warm-start the first keyframe. This needs to be in accordance with the absolute pose we give to the landmark as initial specification.

In other words, if a landmark is at a particular location, before you see it it is impossible to know at which location is the camera.

Therefore the first keyframe needs to be created after analysing the first image and deciding which landmark receives the prior.

Once the landmark is positioned, we need to compose its pose with the measurement of the tag, to conclude on the pose of the camera.

Only then the first landmark can be created.

